***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [ ]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME            PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
assign3cluster  GCE       4                                       RUNNING  us-central1-a


# Imports & Setup

In [ ]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [ ]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Dec  7 19:01 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [ ]:
spark

In [ ]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'assign3bucketbgu' 
client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    print(b.name)

graphframes.sh


***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

In [ ]:
def generate_graph(pages):  
  edges = pages.flatMap(lambda x: [Row(x[0],i[0]) for i in x[1]]).distinct()
  vertices = pages.flatMap(lambda x: [Row(x[0])] + [Row(i[0]) for i in x[1]]).distinct()

  return edges, vertices

In [ ]:
t_start = time()
pages_links = spark.read.parquet("gs://wikidata_preprocessed/*").select("id", "anchor_text").rdd
# construct the graph 
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv(f'gs://{bucket_name}/pr', compression="gzip")
pr_time = time() - t_start
pr.show()

+-------+------------------+
|     id|          pagerank|
+-------+------------------+
|3434750| 9917.889528786589|
|  10568|5387.6094700961285|
|  32927| 5284.298441243099|
|  30680|5130.3860058361915|
|5843419| 4959.648354746549|
|  68253| 4771.279909629093|
|  31717| 4488.233081199126|
|  11867| 4148.154882176616|
|  14533|3998.1437395714433|
| 645042|3533.1092975401475|
|  17867| 3247.460763251462|
|5042916|  2993.20144510815|
|4689264| 2983.576551154408|
|  14532|2935.9785290242507|
|  25391|2904.7648285829773|
|   5405|2892.6298433630345|
|4764461|2835.5565581932315|
|  15573|2785.0334936159065|
|   9316|2783.2072557286956|
|8569916|2776.4509667609473|
+-------+------------------+
only showing top 20 rows



In [ ]:
# test that PageRank computaion took less than 1 hour
assert pr_time < 60*60